# Path Stats Over Time

In [1]:
import sys
sys.path.append("..")

import os
from google.cloud import bigquery
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from src import SankeyFlow
from src import Flow




credential_path = "/home/kerri/bigquery-jaya-consultant-cosmic-octane-88917-c46ba9b53a3b.json"
assert os.path.exists(credential_path)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path
project_id = 'cosmic-octane-88917'
client = bigquery.Client(project=project_id)

In [2]:
FlowName = "United Kingdom - Customer Service"
flow = Flow(FlowName)
flow.top_paths_plot()

FileNotFoundError: [Errno 2] No such file or directory: '/home/kerri/Documents/Projects/intelepeer/sharkninja_analysis/analysis_modules/src/Flow/top_paths.sql'

In [ ]:
sql_time_evol = """
WITH count_steps AS (
SELECT
        TimeStamp,
        SessionId,
        EndTime - StartTime AS Duration,
        FlowName,
        Path,
        CHARACTER_LENGTH(Path) - CHARACTER_LENGTH(REPLACE(Path, ';', '')) AS pathLength
FROM `cosmic-octane-88917.analytics_us._VW_ApplicationExitEvent`
),

top_10 AS (
SELECT *,
      CONCAT(ROW_NUMBER() OVER(), '-Path_Freq_Rank') AS nickname
FROM (
      SELECT 
        Path, 
        COUNT(TimeStamp) count
      FROM `cosmic-octane-88917.analytics_us._VW_ApplicationExitEvent`
      WHERE FlowName in {0}
      AND EXTRACT(DATE FROM TimeStamp) BETWEEN '2020-06-01' AND '2020-10-31'
      GROUP BY Path
      ORDER BY count DESC
      LIMIT 10
      )
),

calc_values AS (
SELECT 
      EXTRACT(DATE FROM TimeStamp) AS date,
      COUNT(DISTINCT(SessionId)) AS count,
      AVG(Duration) AS avg_duration,
      nickname,
      ANY_VALUE(Path) AS Path
FROM count_steps c
INNER JOIN 
top_10 t
USING(Path)
GROUP BY date, nickname

)

SELECT *,
      AVG(count) OVER(PARTITION BY nickname ORDER BY date 
                 ROWS BETWEEN 14 PRECEDING AND CURRENT ROW) AS avg_14_day_count,
      AVG(avg_duration) OVER(PARTITION BY nickname ORDER BY date 
                 ROWS BETWEEN 14 PRECEDING AND CURRENT ROW) AS avg_14_day_duration
FROM calc_values

"""


In [ ]:


def time_stats(df, hue):
    
    
    fig, (ax1,ax2, ax3, ax4) = plt.subplots(nrows=4, figsize=(15,30))
    chart = sns.lineplot(x="date", y="avg_14_day_count",
                 hue=hue, 
                 data=df, ax=ax1)
    chart.set_title(f"14 Day Rolling Average Count")


    chart = sns.lineplot(x="date", y="avg_14_day_duration",
                 hue=hue, 
                 data=df, ax=ax2)
    chart.set_title(f"14 Day Rolling Avg Duration")


    chart = sns.lineplot(x="date", y="count",
                 hue=hue, 
                 data=df, ax=ax3)
    chart.set_title(f"Count")


    chart = sns.lineplot(x="date", y="avg_duration",
                 hue=hue, 
                 data=df, ax=ax4)
    chart.set_title(f"Duration")

In [ ]:
FlowName = "United Kingdom - Customer Service"
df = client.query(sql_time_evol.format(f"('{FlowName}')")).to_dataframe()
time_stats(df, 'nickname')


In [ ]:
FlowName = "Italy - Customer Service"
df = client.query(sql_time_evol.format(f"('{FlowName}')")).to_dataframe()
time_stats(df, 'nickname')

In [ ]:
FlowName = "France - Customer Service"
df = client.query(sql_time_evol.format(f"('{FlowName}')")).to_dataframe()
time_stats(df, 'nickname')

In [ ]:
FlowName = "Spain - Customer Service"
df = client.query(sql_time_evol.format(f"('{FlowName}')")).to_dataframe()
time_stats(df, 'nickname')

In [ ]:
FlowName = "Germany - Customer Service"
df = client.query(sql_time_evol.format(f"('{FlowName}')")).to_dataframe()
time_stats(df, 'nickname')

In [ ]:
FlowName = "DB Management"
df = client.query(sql_time_evol.format(f"('{FlowName}')")).to_dataframe()
time_stats(df, 'nickname')

In [ ]:
FlowName = "v2 Main UK Cust Service - Main v2.0"
df = client.query(sql_time_evol.format(f"('{FlowName}')")).to_dataframe()
time_stats(df, 'nickname')

In [ ]:
FlowName = "v2 survey UK Cust Service - Survey Opt In v2.0"
df = client.query(sql_time_evol.format(f"('{FlowName}')")).to_dataframe()
time_stats(df, 'nickname')